# Ultralytics YOLO output

In [ ]:
import ultralytics
ultralytics.__version__

In [ ]:
import os

# Set an environment variable
os.environ['YOLO_VERBOSE'] = "False"

# Access the environment variable
print(os.environ['YOLO_VERBOSE'])


In [3]:
from tqdm import tqdm

def process_mot20_results(files_in_directory, model, output_file="output_mot.txt"):
    mot_results = []

    # Use tqdm to display a progress bar while iterating over frames
    for frame in tqdm(files_in_directory, desc="Processing frames"):
        results = model([frame])  # Run model on the frame

        # Extract the filename and frame number
        filename = os.path.basename(frame)
        frame_number = int(filename.split('_')[1].split('.')[0])

        # Process each result from the model output
        for result in results:
            boxes = result.boxes  # Boxes object for bounding box outputs
            box_data = boxes.xywh  # Get [x_center, y_center, width, height] format
            conf = boxes.conf  # Confidence scores
            cls = boxes.cls  # Class labels (if needed for IDs)

            # Loop through each detected box
            for i, (xywh, score) in enumerate(zip(box_data, conf)):
                bb_left = xywh[0] - (xywh[2] / 2)  # Convert x_center to bb_left
                bb_top = xywh[1] - (xywh[3] / 2)   # Convert y_center to bb_top
                bb_width = xywh[2]
                bb_height = xywh[3]
                obj_id = -1  # Assign a unique ID (or use a tracker to get actual IDs)

                class_id = 1
                # Format: frame, id, bb_left, bb_top, bb_width, bb_height, conf, -1, -1, -1
                # MOT15 frame_number, identity_number, bbox left, bbox_top, bbox_width, bbox_height, confidence, class, visibility
                mot_line = f"{frame_number}, {obj_id}, {bb_left:.2f}, {bb_top:.2f}, {bb_width:.2f}, {bb_height:.2f}, {score:.2f}, {class_id}, 1"
                # MOT15 frame_number, identity_number, bbox left, bbox_top, bbox_width, bbox_height, confidence, x, y, z
                # mot_line = f"{frame_number}, {obj_id}, {bb_left:.2f}, {bb_top:.2f}, {bb_width:.2f}, {bb_height:.2f}, {score:.2f}, -1, -1, -1"

                mot_results.append(mot_line)

    # Write results to a .txt file in MOT format
    with open(output_file, "w") as f:
        f.write("\n".join(mot_results))
    print(f"Results saved to {output_file}")

In [4]:
def save_metrics_to_txt(metrics_dict, filename="metrics_results.txt"):
    """
    Save the given dictionary of metrics to a text file.
    
    Args:
    - metrics_dict (dict): Dictionary containing metric names as keys and their values.
    - filename (str): The name of the file to save the metrics. Default is "metrics_results.txt".
    """
    with open(filename, "w") as f:
        for metric, value in metrics_dict.items():
            f.write(f"{metric}: {value}\n")
    
    print(f"Metrics saved to {filename}")

In [5]:

def process_mot20_results(files_in_directory, model, output_file="output_mot.txt"):
    mot_results = []

    # Use tqdm to display a progress bar while iterating over frames
    for frame in tqdm(files_in_directory, desc="Processing frames"):
        results = model([frame])  # Run model on the frame

        # Extract the filename and frame number
        filename = os.path.basename(frame)
        frame_number = int(filename.split('_')[1].split('.')[0])

        # Process each result from the model output
        for result in results:
            boxes = result.boxes  # Boxes object for bounding box outputs
            box_data = boxes.xyxy  # Get [x_center, y_center, width, height] format
            conf = boxes.conf  # Confidence scores
            cls = boxes.cls  # Class labels (if needed for IDs)

            # Loop through each detected box
            for i, (xyxy, score) in enumerate(zip(box_data, conf)):
                bb_left = xyxy[0]  # Convert x_center to bb_left
                bb_top = xyxy[1]  # Convert y_center to bb_top
                bb_width = xyxy[2]-xyxy[0]
                bb_height = xyxy[3]-xyxy[1]
                obj_id = -1  # Assign a unique ID (or use a tracker to get actual IDs)

                class_id = 1
                # Format: frame, id, bb_left, bb_top, bb_width, bb_height, conf, -1, -1, -1
                # MOT15 frame_number, identity_number, bbox left, bbox_top, bbox_width, bbox_height, confidence, class, visibility
                #mot_line = f"{frame_number}, {obj_id}, {bb_left:.2f}, {bb_top:.2f}, {bb_width:.2f}, {bb_height:.2f}, {score:.2f}, {class_id}, 1"
                # MOT15 frame_number, identity_number, bbox left, bbox_top, bbox_width, bbox_height, confidence, x, y, z
                mot_line = f"{frame_number}, {obj_id}, {bb_left:.2f}, {bb_top:.2f}, {bb_width:.2f}, {bb_height:.2f}, {score:.2f}, -1, -1, -1"

                mot_results.append(mot_line)

    # Write results to a .txt file in MOT format
    with open(output_file, "w") as f:
        f.write("\n".join(mot_results))
    print(f"Results saved to {output_file}")

In [6]:
from ultralytics import YOLO, RTDETR
# Load the weights of the YOLO model
model_name = "yolo11x_DJI_0008_V@fine-tuning"

model = YOLO(f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{model_name}/best.pt", verbose=False)
#model = RTDETR(f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{model_name}/best.pt")

In [7]:
import os

def list_files(directory):
    try:
        # List all files in the given directory
        files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
        return files
    except Exception as e:
        return str(e)  # Return the error message as a string

def list_directories(path='.'):
    return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

## Validation

In [8]:
video_name = "DJI_20240308111117_0010_V_1"

# Example usage:
directory_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{video_name}/valid/images"

files_in_directory = [os.path.join(directory_path, file) for file in list_files(directory_path)]

In [ ]:
run_name = model_name + "@" + video_name

data_yaml_file = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{video_name}/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308111117_0010_V_2/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308110958_0008_V_1/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308110958_0008_V_2/data.yaml"

validation_results = model.val(
    data=data_yaml_file, imgsz=640, batch=16, conf=0.25, iou=0.6, 
    device="cpu", save_json=True, name=f"{run_name}")

In [ ]:
save_metrics_to_txt(validation_results.results_dict, filename="./"+ str(validation_results.save_dir) + "/output.txt")

In [ ]:
OUTPATH_MOT_FILES = f'../output/mot_detections/{run_name}'

# Create the directory
try:
    os.makedirs(OUTPATH_MOT_FILES, exist_ok=True)  # Creates intermediate directories if needed
    print(f"Directory created at: {OUTPATH_MOT_FILES}")
except OSError as e:
    print(f"Error creating directory: {e}")

output_mot_file = os.path.join(OUTPATH_MOT_FILES, "output.txt")
process_mot20_results(files_in_directory, model, output_file=output_mot_file)

In [14]:
video_name = "DJI_20240308110228_0006_V_2"

# Example usage:
directory_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{video_name}/valid/images"

files_in_directory = [os.path.join(directory_path, file) for file in list_files(directory_path)]

In [ ]:
video_name = "DJI_20240308110228_0006_V_2"
run_name = model_name + "@" + video_name

data_yaml_file = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{video_name}/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308111117_0010_V_2/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308110958_0008_V_1/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308110958_0008_V_2/data.yaml"

validation_results = model.val(
    data=data_yaml_file, imgsz=640, batch=16, conf=0.25, iou=0.6, 
    device="cpu", save_json=True, name=f"{run_name}")

In [ ]:
save_metrics_to_txt(validation_results.results_dict, filename="./"+ str(validation_results.save_dir) + "/output.txt")

In [ ]:
OUTPATH_MOT_FILES = f'../output/mot_detections/{run_name}'

# Create the directory
try:
    os.makedirs(OUTPATH_MOT_FILES, exist_ok=True)  # Creates intermediate directories if needed
    print(f"Directory created at: {OUTPATH_MOT_FILES}")
except OSError as e:
    print(f"Error creating directory: {e}")

output_mot_file = os.path.join(OUTPATH_MOT_FILES, "output.txt")
process_mot20_results(files_in_directory, model, output_file=output_mot_file)

In [18]:
video_name = "DJI_20240308110115_0005_V"

# Example usage:
directory_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{video_name}/valid/images"

files_in_directory = [os.path.join(directory_path, file) for file in list_files(directory_path)]

In [ ]:
video_name = "DJI_20240308110115_0005_V"
run_name = model_name + "@" + video_name

data_yaml_file = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{video_name}/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308111117_0010_V_2/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308110958_0008_V_1/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308110958_0008_V_2/data.yaml"

validation_results = model.val(
    data=data_yaml_file, imgsz=640, batch=16, conf=0.25, iou=0.6, 
    device="cpu", save_json=True, name=f"{run_name}")

In [ ]:
save_metrics_to_txt(validation_results.results_dict, filename="./"+ str(validation_results.save_dir) + "/output.txt")

In [ ]:
OUTPATH_MOT_FILES = f'../output/mot_detections/{run_name}'

# Create the directory
try:
    os.makedirs(OUTPATH_MOT_FILES, exist_ok=True)  # Creates intermediate directories if needed
    print(f"Directory created at: {OUTPATH_MOT_FILES}")
except OSError as e:
    print(f"Error creating directory: {e}")

output_mot_file = os.path.join(OUTPATH_MOT_FILES, "output.txt")
process_mot20_results(files_in_directory, model, output_file=output_mot_file)

In [22]:
video_name = "DJI_20240308110013_0004_V_1"

# Example usage:
directory_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{video_name}/valid/images"

files_in_directory = [os.path.join(directory_path, file) for file in list_files(directory_path)]

In [ ]:
video_name = "DJI_20240308110013_0004_V_1"
run_name = model_name + "@" + video_name

data_yaml_file = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{video_name}/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308111117_0010_V_2/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308110958_0008_V_1/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308110958_0008_V_2/data.yaml"

validation_results = model.val(
    data=data_yaml_file, imgsz=640, batch=16, conf=0.25, iou=0.6, 
    device="cpu", save_json=True, name=f"{run_name}")

In [ ]:
save_metrics_to_txt(validation_results.results_dict, filename="./"+ str(validation_results.save_dir) + "/output.txt")

In [ ]:
OUTPATH_MOT_FILES = f'../output/mot_detections/{run_name}'

# Create the directory
try:
    os.makedirs(OUTPATH_MOT_FILES, exist_ok=True)  # Creates intermediate directories if needed
    print(f"Directory created at: {OUTPATH_MOT_FILES}")
except OSError as e:
    print(f"Error creating directory: {e}")

output_mot_file = os.path.join(OUTPATH_MOT_FILES, "output.txt")
process_mot20_results(files_in_directory, model, output_file=output_mot_file)

In [26]:
video_name = "DJI_20240308110013_0004_V_3"

# Example usage:
directory_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{video_name}/valid/images"

files_in_directory = [os.path.join(directory_path, file) for file in list_files(directory_path)]

In [ ]:
video_name = "DJI_20240308110013_0004_V_3"
run_name = model_name + "@" + video_name

data_yaml_file = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{video_name}/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308111117_0010_V_2/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308110958_0008_V_1/data.yaml"
#data_yaml_file = "C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/DJI_20240308110958_0008_V_2/data.yaml"

validation_results = model.val(
    data=data_yaml_file, imgsz=640, batch=16, conf=0.25, iou=0.6, 
    device="cpu", save_json=True, name=f"{run_name}")

In [ ]:
save_metrics_to_txt(validation_results.results_dict, filename="./"+ str(validation_results.save_dir) + "/output.txt")

In [ ]:
OUTPATH_MOT_FILES = f'../output/mot_detections/{run_name}'

# Create the directory
try:
    os.makedirs(OUTPATH_MOT_FILES, exist_ok=True)  # Creates intermediate directories if needed
    print(f"Directory created at: {OUTPATH_MOT_FILES}")
except OSError as e:
    print(f"Error creating directory: {e}")

output_mot_file = os.path.join(OUTPATH_MOT_FILES, "output.txt")
process_mot20_results(files_in_directory, model, output_file=output_mot_file)